In [1]:
import sys
sys.path.append('../../../')

In [2]:
import pandas as pd
import numpy as np

from modules import machine_learning_utils as mlu

pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

# 1.0 Data retrieval

In [3]:
df = pd.read_pickle("../../../data_lake/output/df_under.pkl")

In [4]:
df.sample(2)

,TransactionID,isFraud,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist2,P_emaildomain,R_emaildomain,M4,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,max_c,max_d,customer_id,num_transaction_per_time,multi_transaction_per_time
88387,3283652,0,25.000,H,6419,543.0,150.0,mastercard,224.0,debit,472.0,87.0,NaN,gmail.com,gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,iOS 11.2.6,mobile safari generic,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device,1.0,0.000000,7900,1,0
7214,3013723,0,18.368,C,13413,103.0,185.0,visa,137.0,credit,511.0,60.0,649.0,hotmail.com,hotmail.com,M0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,NaN,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows,1.0,252.833328,4646,1,0


In [5]:
df.shape

(144233, 41)

# 2.0 Data Partitioning

In [6]:
df['isFraud'].value_counts()

0    132915
1     11318
Name: isFraud, dtype: int64

## 2.1 Train, validation, test split

In [7]:
train, val, test = mlu.partitioning(df,0.7,0.15,'isFraud')

Train shape:  (100963, 41)
Train %: 
 0    0.920496
1    0.079504
Name: isFraud, dtype: float64
Val %: 
 0    0.923319
1    0.076681
Name: isFraud, dtype: float64
Test %: 
 0    0.924567
1    0.075433
Name: isFraud, dtype: float64


## 2.2 Adversarial validation between val and test set

In [8]:
drop_cols = ['TransactionID','customer_id']

In [9]:
mlu.adversarial_validation(val,test,drop_cols)

No distribution shift, OK! AUC is:  0.5046975542178611


## 2.3 Class imbalance

In [10]:
final_train = mlu.class_imbalance(train,'isFraud')

0    8027
1    8027
Name: isFraud, dtype: int64
Train dataset shape:  (16054, 41)


# 3.0 Output

In [11]:
final_train.to_pickle('../../../data_lake/output/train.pkl')
val.to_pickle('../../../data_lake/output/val.pkl')
test.to_pickle('../../../data_lake/output/test.pkl')